## Mã hóa số nguyên

Mã hóa số nguyên gồm việc thay thế các hạng mục bằng các chữ số từ 1 đến n (hoặc 0 đến n-1, tùy thuộc vào cách triển khai), trong đó n là số hạng mục riêng biệt của biến.

Các con số được chỉ định tùy ý. Phương pháp mã hóa này cho phép xác định đối chuẩn (benchmarking) nhanh chóng của các mô hình học máy.


### Ưu điểm

- Dễ triển khai.
- Không mở rộng không gian đặc trưng.


### Hạn chế

- Không thu thập thông tin về các nhãn hạng mục.
- Không thích hợp với mô hình tuyến tính.

Mã hóa số nguyên thích hợp hơn với các phương pháp phi tuyến tính có khả năng điều hướng thông qua các chữ số được chỉ định tùy ý để tìm những mẫu liên kết chúng với mục tiêu.


## Trong bản mô phỏng này:

Chúng ta sẽ thực hiện mã hóa one-hot với:
- pandas
- Scikit-learn
- Feature-Engine

Chúng ta sẽ sử dụng tập dữ liệu giá nhà để minh họa các ưu điểm và hạn chế của từng triển khai.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!pip install feature_engine

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 326.6/326.6 kB 4.7 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd

# chia tập dữ liệu
from sklearn.model_selection import train_test_split

# mã hóa số nguyên sử dụng sklearn
from sklearn.preprocessing import LabelEncoder

# mã hóa số nguyên sử dụng feature-engine
from feature_engine.encoding import OrdinalEncoder

In [3]:
# load tập dữ liệu

data = pd.read_csv(
    '/content/drive/MyDrive/Colab_Notebooks/Bai tap lap/Mon_hoc_2/house-price/houseprice.csv',
    usecols=['Neighborhood', 'Exterior1st', 'Exterior2nd', 'SalePrice'])

data.head()

,Neighborhood,Exterior1st,Exterior2nd,SalePrice
0,CollgCr,VinylSd,VinylSd,208500
1,Veenker,MetalSd,MetalSd,181500
2,CollgCr,VinylSd,VinylSd,223500
3,Crawfor,Wd Sdng,Wd Shng,140000
4,NoRidge,VinylSd,VinylSd,250000


In [4]:
# xem từng biến có bao nhiêu nhãn

for col in data.columns:
    print(col, ': ', len(data[col].unique()), ' labels')

Neighborhood :  25  labels
Exterior1st :  15  labels
Exterior2nd :  16  labels
SalePrice :  663  labels


In [5]:
# khám phá các hạng mục duy nhất
data['Neighborhood'].unique()

array(['CollgCr', 'Veenker', 'Crawfor', 'NoRidge', 'Mitchel', 'Somerst',
       'NWAmes', 'OldTown', 'BrkSide', 'Sawyer', 'NridgHt', 'NAmes',
       'SawyerW', 'IDOTRR', 'MeadowV', 'Edwards', 'Timber', 'Gilbert',
       'StoneBr', 'ClearCr', 'NPkVill', 'Blmngtn', 'BrDale', 'SWISU',
       'Blueste'], dtype=object)

In [6]:
data['Exterior1st'].unique()

array(['VinylSd', 'MetalSd', 'Wd Sdng', 'HdBoard', 'BrkFace', 'WdShing',
       'CemntBd', 'Plywood', 'AsbShng', 'Stucco', 'BrkComm', 'AsphShn',
       'Stone', 'ImStucc', 'CBlock'], dtype=object)

In [7]:
data['Exterior2nd'].unique()

array(['VinylSd', 'MetalSd', 'Wd Shng', 'HdBoard', 'Plywood', 'Wd Sdng',
       'CmentBd', 'BrkFace', 'Stucco', 'AsbShng', 'Brk Cmn', 'ImStucc',
       'AsphShn', 'Stone', 'Other', 'CBlock'], dtype=object)

### Mã hóa quan trọng

Cần lựa chọn số để gán cho từng hạng mục trong tập huấn luyện, rồi sử dụng các mapping đó trong tập kiểm tra.

In [8]:
# hãy chia thành tập huấn luyện và tập kiểm tra

X_train, X_test, y_train, y_test = train_test_split(
    data[['Neighborhood', 'Exterior1st', 'Exterior2nd']], # các yếu tố dự báo
    data['SalePrice'],  # mục tiêu
    test_size=0.3,  # phần trăm các quan sát trong tập kiểm tra
    random_state=0)  # seed đảm bảo khả năng tái lặp

X_train.shape, X_test.shape

((1022, 3), (438, 3))

## Mã hóa số nguyên với pandas


### Ưu điểm

- nhanh
- trả về pandas dataframe

### Hạn chế của pandas:

- Không giữ thông tin từ tập huấn luyện để truyền tới dữ liệu kiểm tra.

Chúng ta cần thu nạp và lưu từng mapping cách thủ công nếu định dùng chúng trong sản xuất.

In [9]:
## Yêu cầu 1: trước tiên tạo một dictionary có mapping của các hạng mục

## VIẾT CODE Ở ĐÂY:
ordinal_mapping = {
    k: i
    for i, k in enumerate(X_train['Neighborhood'].unique(), 0)
}

ordinal_mapping

{'CollgCr': 0,
 'ClearCr': 1,
 'BrkSide': 2,
 'Edwards': 3,
 'SWISU': 4,
 'Sawyer': 5,
 'Crawfor': 6,
 'NAmes': 7,
 'Mitchel': 8,
 'Timber': 9,
 'Gilbert': 10,
 'Somerst': 11,
 'MeadowV': 12,
 'OldTown': 13,
 'BrDale': 14,
 'NWAmes': 15,
 'NridgHt': 16,
 'SawyerW': 17,
 'NoRidge': 18,
 'IDOTRR': 19,
 'NPkVill': 20,
 'StoneBr': 21,
 'Blmngtn': 22,
 'Veenker': 23,
 'Blueste': 24}

<details><summary> Gợi ý </summary>

[unique()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.unique.html)

</details>

Dictionary cho biết số nào sẽ thay cho từng hạng mục. Các số được chỉ định tùy ý từ 0 đến n-1, trong đó n là số hạng mục riêng biệt.

In [10]:
## Yêu cầu 2: thay các nhãn bằng các số nguyên

## VIẾT CODE Ở ĐÂY:
X_train['Neighborhood'] = X_train['Neighborhood'].map(ordinal_mapping)
X_test['Neighborhood'] = X_test['Neighborhood'].map(ordinal_mapping)

In [11]:

# khám phá kết quả

X_train['Neighborhood'].head(10)

64      0
682     1
960     2
1384    3
1100    4
416     5
1034    6
853     7
472     3
1011    3
Name: Neighborhood, dtype: int64

In [13]:
# chuyển các lệnh trước đó thành 2 hàm


def find_category_mappings(df, variable):
    return {k: i for i, k in enumerate(df[variable].unique(), 0)}


def integer_encode(train, test, variable, ordinal_mapping):

    X_train[variable] = X_train[variable].map(ordinal_mapping)
    X_test[variable] = X_test[variable].map(ordinal_mapping)
    return X_train[variable], X_test[variable]

In [14]:
## Yêu cầu 3: chạy một vòng lặp qua các biến hạng mục còn lại

for variable in ['Exterior1st', 'Exterior2nd']:
    ## VIẾT CODE Ở ĐÂY:
    mappings = find_category_mappings(X_train, variable)
    X_train[variable], X_test[variable] = integer_encode(X_train, X_test, variable, mappings)

In [15]:
# hãy xem kết quả

X_train.head()

,Neighborhood,Exterior1st,Exterior2nd
64,0,0,0
682,1,1,1
960,2,1,2
1384,3,2,3
1100,4,1,1


## Mã hóa số nguyên với Scikit-learn

In [16]:
# hãy chia thành tập huấn luyện và tập kiểm tra

X_train, X_test, y_train, y_test = train_test_split(
    data[['Neighborhood', 'Exterior1st', 'Exterior2nd']], # các yếu tố dự báo
    data['SalePrice'],  # mục tiêu
    test_size=0.3,  # phần trăm các quan sát trong tập kiểm tra
    random_state=0)  # seed đảm bảo khả năng tái lặp

X_train.shape, X_test.shape

((1022, 3), (438, 3))

In [20]:
# tạo một encoder

le = LabelEncoder()
le.fit(X_train['Neighborhood'])

LabelEncoder()

In [21]:
# có thể thấy các class duy nhất

le.classes_

array(['Blmngtn', 'Blueste', 'BrDale', 'BrkSide', 'ClearCr', 'CollgCr',
       'Crawfor', 'Edwards', 'Gilbert', 'IDOTRR', 'MeadowV', 'Mitchel',
       'NAmes', 'NPkVill', 'NWAmes', 'NoRidge', 'NridgHt', 'OldTown',
       'SWISU', 'Sawyer', 'SawyerW', 'Somerst', 'StoneBr', 'Timber',
       'Veenker'], dtype=object)

In [22]:
X_train['Neighborhood'] = le.transform(X_train['Neighborhood'])
X_test['Neighborhood'] = le.transform(X_test['Neighborhood'])

X_train.head()

,Neighborhood,Exterior1st,Exterior2nd
64,5,VinylSd,VinylSd
682,4,Wd Sdng,Wd Sdng
960,3,Wd Sdng,Plywood
1384,7,WdShing,Wd Shng
1100,18,Wd Sdng,Wd Sdng


Không may, LabelEncoder hoạt động với một biến tại thời điểm đó. Tuy nhiên, có một cách tự động điều này cho tất cả các biến hạng mục. Xem [stackoverflow thread](https://stackoverflow.com/questions/24458645/label-encoding-across-multiple-columns-in-scikit-learn).

In [23]:
# yêu cầu import bổ sung

from collections import defaultdict

In [24]:
# hãy chia thành tập huấn luyện và tập kiểm tra

X_train, X_test, y_train, y_test = train_test_split(
    data[['Neighborhood', 'Exterior1st', 'Exterior2nd']], # các yếu tố dự báo
    data['SalePrice'],  # mục tiêu
    test_size=0.3,  # phần trăm các quan sát trong tập kiểm tra
    random_state=0)  # seed đảm bảo khả năng tái lặp

X_train.shape, X_test.shape

((1022, 3), (438, 3))

In [25]:
d = defaultdict(LabelEncoder)

In [26]:
# mã hóa biến
train_transformed = X_train.apply(lambda x: d[x.name].fit_transform(x))

# # Sử dụng dictionary để mã hóa dữ liệu tương lai
test_transformed = X_test.apply(lambda x: d[x.name].transform(x))

In [27]:
train_transformed.head()

,Neighborhood,Exterior1st,Exterior2nd
64,5,12,13
682,4,13,14
960,3,13,10
1384,7,14,15
1100,18,13,14


In [28]:
test_transformed.head()

,Neighborhood,Exterior1st,Exterior2nd
529,6,13,11
491,12,13,14
459,3,8,8
279,4,9,10
655,2,6,7


In [29]:
# và nghịch đảo biến đổi để khôi phục các nhãn ban đầu

# # nghịch đảo đã mã hóa
tmp = train_transformed.apply(lambda x: d[x.name].inverse_transform(x))
tmp.head()

,Neighborhood,Exterior1st,Exterior2nd
64,CollgCr,VinylSd,VinylSd
682,ClearCr,Wd Sdng,Wd Sdng
960,BrkSide,Wd Sdng,Plywood
1384,Edwards,WdShing,Wd Shng
1100,SWISU,Wd Sdng,Wd Sdng


Cuối cùng, có một transformer Scikit-learn khác để mã hóa nhiều biến cùng một lúc: OrdinalEncoder. Tuy nhiên, transformer này trả về một mảng NumPy không có tên cột nên nó không phải cách triển khai yêu thích của tôi. Chi tiết tại: https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OrdinalEncoder.html.

## Mã hóa số nguyên với Feature-Engine

In [30]:
# chia thành tập huấn luyện và tập kiểm tra

X_train, X_test, y_train, y_test = train_test_split(
    data[['Neighborhood', 'Exterior1st', 'Exterior2nd']], # các yếu tố dự báo
    data['SalePrice'],  # mục tiêu
    test_size=0.3,  # phần trăm các quan sát trong tập kiểm tra
    random_state=0)  # seed đảm bảo khả năng tái lặp

X_train.shape, X_test.shape

((1022, 3), (438, 3))

In [31]:
ordinal_enc = OrdinalEncoder(
    encoding_method='arbitrary',
    variables=['Neighborhood', 'Exterior1st', 'Exterior2nd'])

ordinal_enc.fit(X_train)

OrdinalEncoder(encoding_method='arbitrary',
               variables=['Neighborhood', 'Exterior1st', 'Exterior2nd'])

In [32]:
# trong dict encoder, chúng ta thấy các số
# được chỉ định cho từng hạng mục cho tất cả các biến được chỉ ra

ordinal_enc.encoder_dict_

{'Neighborhood': {'CollgCr': 0,
  'ClearCr': 1,
  'BrkSide': 2,
  'Edwards': 3,
  'SWISU': 4,
  'Sawyer': 5,
  'Crawfor': 6,
  'NAmes': 7,
  'Mitchel': 8,
  'Timber': 9,
  'Gilbert': 10,
  'Somerst': 11,
  'MeadowV': 12,
  'OldTown': 13,
  'BrDale': 14,
  'NWAmes': 15,
  'NridgHt': 16,
  'SawyerW': 17,
  'NoRidge': 18,
  'IDOTRR': 19,
  'NPkVill': 20,
  'StoneBr': 21,
  'Blmngtn': 22,
  'Veenker': 23,
  'Blueste': 24},
 'Exterior1st': {'VinylSd': 0,
  'Wd Sdng': 1,
  'WdShing': 2,
  'HdBoard': 3,
  'MetalSd': 4,
  'AsphShn': 5,
  'BrkFace': 6,
  'Plywood': 7,
  'CemntBd': 8,
  'Stucco': 9,
  'BrkComm': 10,
  'AsbShng': 11,
  'ImStucc': 12,
  'CBlock': 13,
  'Stone': 14},
 'Exterior2nd': {'VinylSd': 0,
  'Wd Sdng': 1,
  'Plywood': 2,
  'Wd Shng': 3,
  'HdBoard': 4,
  'MetalSd': 5,
  'AsphShn': 6,
  'CmentBd': 7,
  'BrkFace': 8,
  'Stucco': 9,
  'ImStucc': 10,
  'Stone': 11,
  'AsbShng': 12,
  'Brk Cmn': 13,
  'CBlock': 14,
  'Other': 15}}

In [33]:
# list các biến mà encoder sẽ biến đổi

ordinal_enc.variables_

['Neighborhood', 'Exterior1st', 'Exterior2nd']

In [34]:
X_train = ordinal_enc.transform(X_train)
X_test = ordinal_enc.transform(X_test)

# hãy khám phá kết quả
X_train.head()

,Neighborhood,Exterior1st,Exterior2nd
64,0,0,0
682,1,1,1
960,2,1,2
1384,3,2,3
1100,4,1,1


**Lưu ý**

Nếu các biến đối số được để thành None thì encoder sẽ tự động xác định tất cả các biến hạng mục. Tuyệt đúng không?

Encoder sẽ không mã hóa các biến dạng số. Vì vậy, nếu một số biến dạng số là biến hạng mục thì chúng ta sẽ cần ép kiểu lại chúng thành object trước khi sử dụng encoder.

Nếu không có biến trong tập kiểm tra mà encoder không có số để chỉ định (không thấy hạng mục trong tập huấn luyện) thì encoder sẽ trả về lỗi.